In [41]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader


In [42]:
NUM_EPOCHS = 5
nic = pd.read_csv('stevilka0.csv').dropna()
ena = pd.read_csv('stevilka1.csv').dropna()

nic.iloc[0]
data = pd.concat([nic, ena]) #združim


ndArray = data.to_numpy()
tensor = torch.from_numpy(ndArray)
print(ndArray[0])
print(tensor)


[ -99. 2639. 3181.  -19.  -80.  -41.   72. -166. -428.]
tensor([[  -99.,  2639.,  3181.,  ...,    72.,  -166.,  -428.],
        [  730.,  -804.,  4397.,  ...,   -36.,   167.,  -266.],
        [ -224., -1672.,  4828.,  ...,   143.,    45.,    65.],
        ...,
        [ -682., -1208.,  4079.,  ...,   100.,    89.,  -232.],
        [ -219., -1849.,  4827.,  ...,   151.,    93.,  -260.],
        [ -565., -1557.,  3998.,  ...,   220.,   108.,  -287.]],
       dtype=torch.float64)


In [48]:
class AirWrite(nn.Module):
    def __init__(self):
        super(AirWrite, self).__init__()
        self.fc1 = nn.Linear(9, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

In [57]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

model = AirWrite().to(device)
loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

def train():
    #labels = labels.unsqueeze(1) 
    dataloader = TensorDataset(inputs, labels)
    train_loader = DataLoader(tensor, batch_size = 120, shuffle = True)

    running_loss = 0.0
    total = 0
    correct = 0

    model.train()
    for epoch in range (NUM_EPOCHS):
        for i, data in enumerate(train_loader):

            inputs, labels = inputs.to(device), labels.to(device).unsqueeze(1)
            
            optimizer.zero_grad()
            outputs = model(inputs)

            loss = loss_fn(outputs, labels)
            loss.backward() 
            optimizer.step()  

            running_loss += loss.item() * inputs.size(0)
            predicted = outputs.round()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / total
        epoch_accuracy = correct / total
        print(f'Epoch {epoch+1}/{NUM_EPOCHS}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')
        print(device)

cuda


In [58]:
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)
    model = train()
    torch.save(model, "airWriting_model.pth")

cuda


RuntimeError: CUDA error: CUBLAS_STATUS_EXECUTION_FAILED when calling `cublasLtMatmul( ltHandle, computeDesc.descriptor(), &alpha_val, mat1_ptr, Adesc.descriptor(), mat2_ptr, Bdesc.descriptor(), &beta_val, result_ptr, Cdesc.descriptor(), result_ptr, Cdesc.descriptor(), &heuristicResult.algo, workspace.data_ptr(), workspaceSize, at::cuda::getCurrentCUDAStream())`